# Quantitative Value Investment Strategy
"Value investing" means investing in the stocks that are cheapest relative to common measures of business value (like earnings or assets).

For this project, we're going to build an investing strategy that selects the 50 stocks with the best value metrics. From there, we will calculate recommended trades for an equal-weight portfolio of these 50 stocks.

## Library Imports

In [1]:
import numpy as np 
import pandas as pd 
import requests
from scipy import stats
import math
import warnings
from api import pub
warnings.filterwarnings('ignore')

# 1. Easy version of QVIS

## Importing list of stocks from wikipedia

In [2]:
tickers = pd.read_html(
    'https://en.wikipedia.org/wiki/List_of_S%26P_500_companies')[0]
stocks = pd.DataFrame({'Ticker':tickers['Symbol']})
stocks

,Ticker
0,MMM
1,AOS
2,ABT
3,ABBV
4,ACN
...,...
498,YUM
499,ZBRA
500,ZBH
501,ZION


## Filling out the dataframe with API data

In [3]:
def chunks(lst, n): # converts list to list of chunks
    for i in range(0, len(lst), n):
        yield lst[i:i + n]

In [4]:
# Dividing tickers by 100 to iterate batch requests

symbol_groups = list(chunks(stocks['Ticker'], 100))
symbol_strings = []
for i in range(len(symbol_groups)):
    symbol_strings.append(','.join(symbol_groups[i]))

symbol_strings

['MMM,AOS,ABT,ABBV,ACN,ADBE,AMD,AES,AFL,A,APD,ABNB,AKAM,ALB,ARE,ALGN,ALLE,LNT,ALL,GOOGL,GOOG,MO,AMZN,AMCR,AEE,AAL,AEP,AXP,AIG,AMT,AWK,AMP,AME,AMGN,APH,ADI,ANSS,AON,APA,AAPL,AMAT,APTV,ACGL,ADM,ANET,AJG,AIZ,T,ATO,ADSK,ADP,AZO,AVB,AVY,AXON,BKR,BALL,BAC,BK,BBWI,BAX,BDX,BRK.B,BBY,BIO,TECH,BIIB,BLK,BX,BA,BKNG,BWA,BXP,BSX,BMY,AVGO,BR,BRO,BF.B,BLDR,BG,CDNS,CZR,CPT,CPB,COF,CAH,KMX,CCL,CARR,CTLT,CAT,CBOE,CBRE,CDW,CE,COR,CNC,CNP,CDAY',
 'CF,CHRW,CRL,SCHW,CHTR,CVX,CMG,CB,CHD,CI,CINF,CTAS,CSCO,C,CFG,CLX,CME,CMS,KO,CTSH,CL,CMCSA,CMA,CAG,COP,ED,STZ,CEG,COO,CPRT,GLW,CTVA,CSGP,COST,CTRA,CCI,CSX,CMI,CVS,DHR,DRI,DVA,DE,DAL,XRAY,DVN,DXCM,FANG,DLR,DFS,DG,DLTR,D,DPZ,DOV,DOW,DHI,DTE,DUK,DD,EMN,ETN,EBAY,ECL,EIX,EW,EA,ELV,LLY,EMR,ENPH,ETR,EOG,EPAM,EQT,EFX,EQIX,EQR,ESS,EL,ETSY,EG,EVRG,ES,EXC,EXPE,EXPD,EXR,XOM,FFIV,FDS,FICO,FAST,FRT,FDX,FIS,FITB,FSLR,FE,FI',
 'FLT,FMC,F,FTNT,FTV,FOXA,FOX,BEN,FCX,GRMN,IT,GEHC,GEN,GNRC,GD,GE,GIS,GM,GPC,GILD,GPN,GL,GS,HAL,HIG,HAS,HCA,PEAK,HSIC,HSY,HES,HPE,HLT,HOLX,HD,HON,HRL,HST,HW

In [5]:
# Creating df's base
cols = ['Ticker','Price','Price-to-Earnings Ratio', 'Number of Shares to Buy']
df = pd.DataFrame(columns=cols)

# Batch API request to IEX CLOUD
for index, symbol_string in enumerate(symbol_strings):
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE/{symbol_string}/?token={pub}'
    quote = requests.get(batch_api_call_url).json()
    for s_index, symbol in enumerate(symbol_string.split(',')):
        price = quote[s_index]['latestPrice']
        pe_ratio = quote[s_index]['peRatio']

        new_row = {'Ticker': symbol, 
                   "Price": price, 
                   'Price-to-Earnings Ratio': pe_ratio, 
                   'Number of Shares to Buy': 'N/A'}
        df.loc[len(df)] = new_row

df

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
0,MMM,109.15,-8.16,N/A
1,AOS,82.36,41.81,N/A
2,ABT,110.48,37.45,N/A
3,ABBV,154.90,42.44,N/A
4,ACN,352.88,32.73,N/A
...,...,...,...,...
498,YUM,130.93,24.89,N/A
499,ZBRA,275.60,30.72,N/A
500,ZBH,121.70,53.85,N/A
501,ZION,44.45,4.74,N/A


In [6]:
# Removing glamour stocks and removing stocks with very bad earnings (peRatio = stock price/EPS)
# https://www.ig.com/en/trading-strategies/what-is-a-good-p-e-ratio--181207#:~:text=Very%20low%20vs%20very%20high,a%20PE%20below%20this%20level.

value_stocks = df.sort_values('Price-to-Earnings Ratio')
value_stocks = value_stocks[value_stocks['Price-to-Earnings Ratio']>5][:50]
value_stocks

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
217,GM,36.155,5.09,N/A
25,AAL,13.980,5.22,N/A
13,ALB,149.980,5.32,N/A
174,EQT,39.185,5.37,N/A
299,MPC,147.320,5.51,N/A
122,CMA,56.310,6.41,N/A
181,EG,353.510,6.47,N/A
428,SYF,38.315,7.10,N/A
472,VTRS,10.805,7.16,N/A
100,CF,80.010,7.41,N/A


## Function which shows minimum budget for buying all positions in dataframe

In [7]:
def min_budget(df):
    minimum = math.ceil(np.sum(df['Price']))
    print(f'To own the above portfolio with 1 stock per company you need at least ${minimum}')
    
min_budget(value_stocks)

To own the above portfolio with 1 stock per company you need at least $3947


## Calculating the number of shares with the same monetary amount per company

In [8]:
def calc_num_of_shares(df):

    while True:
        try:
            whole_amount = int(input('Input your portfolio size in dollars -> '))
            break
        except:
            print('Do not input text, only numbers!\n')
    
    amount_on_ticker = whole_amount/len(df.index)
    df['Number of Shares to Buy'] = [math.floor(amount_on_ticker/price) for price in df['Price']]


calc_num_of_shares(value_stocks)
value_stocks

,Ticker,Price,Price-to-Earnings Ratio,Number of Shares to Buy
217,GM,36.155,5.09,553
25,AAL,13.980,5.22,1430
13,ALB,149.980,5.32,133
174,EQT,39.185,5.37,510
299,MPC,147.320,5.51,135
122,CMA,56.310,6.41,355
181,EG,353.510,6.47,56
428,SYF,38.315,7.10,521
472,VTRS,10.805,7.16,1850
100,CF,80.010,7.41,249


## 2. More complex version       
Every valuation metric has certain flaws.

For example, the price-to-earnings ratio doesn't work well with stocks with negative earnings.

Similarly, stocks that buyback their own shares are difficult to value using the price-to-book ratio.

Investors typically use a `composite` basket of valuation metrics to build robust quantitative value strategies. In this section, we will filter for stocks with the lowest percentiles on the following metrics:

* Price-to-earnings ratio
* Price-to-book ratio
* Price-to-sales ratio
* Enterprise Value divided by Earnings Before Interest, Taxes, Depreciation, and Amortization (EV/EBITDA)
* Enterprise Value divided by Gross Profit (EV/GP) 

`rv` - robust value, the name of the strategy

In [9]:
cols = ['Ticker','Price','Number of Shares to Buy', 'Price-to-Earnings Ratio', 'PE Percentile',
         'Price-to-Book Ratio', 'PB Percentile', 'Price-to-Sales Ratio', 'PS Percentile',
         'EV/EBITDA', 'EV/EBITDA Percentile', 'EV/GP', 'EV/GP Percentile',
         'RV Score' ]

In [10]:
rv_df = pd.DataFrame(columns=cols)

# Batch API request to IEX CLOUD
for index, symbol_string in enumerate(symbol_strings):
    batch_api_call_url = f'https://api.iex.cloud/v1/data/core/QUOTE,advanced_stats/{symbol_string}/?token={pub}'
    data = requests.get(batch_api_call_url).json()
    half = int(len(data)/2)
    quote = data[:half]
    adv_stats = data[half:]
    for s_index, symbol in enumerate(symbol_string.split(',')):
        price = quote[s_index]['latestPrice']
        pe_ratio = quote[s_index]['peRatio']
        enterprise_value = adv_stats[s_index]['enterpriseValue']
        ebitda = adv_stats[s_index]['EBITDA']
        gross_profit = adv_stats[s_index]['grossProfit']
        try:
            ev_ebitda = enterprise_value/ebitda
        except:
            ev_ebitda = 'N/A'
        
        try:
            ev_gp = enterprise_value/gross_profit
        except:
            ev_gp = 'N/A'

        new_row = {'Ticker':symbol,'Price':price,
                   'Number of Shares to Buy': 'N/A', 
                   'Price-to-Earnings Ratio': pe_ratio, 'PE Percentile': 'N/A',
                   'Price-to-Book Ratio': adv_stats[s_index]['priceToBook'], 'PB Percentile': 'N/A',
                   'Price-to-Sales Ratio': adv_stats[s_index]['priceToSales'], 'PS Percentile': 'N/A',
                   'EV/EBITDA': ev_ebitda, 'EV/EBITDA Percentile': 'N/A', 
                   'EV/GP': ev_gp, 'EV/GP Percentile': 'N/A',
                   'RV Score': 'N/A'}
        rv_df.loc[len(rv_df)] = new_row
        
rv_df

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,109.15,N/A,-8.16,N/A,12.86,N/A,1.83,N/A,-9.34938,N/A,4.933437,N/A,N/A
1,AOS,82.36,N/A,41.81,N/A,5.36,N/A,2.66,N/A,11.89993,N/A,6.750015,N/A,N/A
2,ABT,110.49,N/A,37.45,N/A,5.09,N/A,4.77,N/A,24.938352,N/A,9.071074,N/A,N/A
3,ABBV,154.87,N/A,42.43,N/A,23.05,N/A,5.06,N/A,12.976413,N/A,8.997987,N/A,N/A
4,ACN,352.80,N/A,32.73,N/A,8.84,N/A,3.65,N/A,18.632979,N/A,10.899937,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498,YUM,130.94,N/A,24.89,N/A,-4.48,N/A,5.20,N/A,19.793725,N/A,8.980833,N/A,N/A
499,ZBRA,275.60,N/A,30.72,N/A,4.70,N/A,2.79,N/A,18.78035,N/A,6.93918,N/A,N/A
500,ZBH,121.71,N/A,53.85,N/A,2.02,N/A,3.48,N/A,15.912433,N/A,5.850316,N/A,N/A
501,ZION,44.45,N/A,4.74,N/A,1.35,N/A,1.32,N/A,3.631114,N/A,1.370346,N/A,N/A


## Feature Engineering

In [11]:
# Deleting rows with nans

rv_df_cleaned = rv_df.dropna(subset=rv_df.columns)
rv_df_cleaned.reset_index(inplace=True)
del rv_df_cleaned['index']
rv_df_cleaned

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,109.15,N/A,-8.16,N/A,12.86,N/A,1.83,N/A,-9.34938,N/A,4.933437,N/A,N/A
1,AOS,82.36,N/A,41.81,N/A,5.36,N/A,2.66,N/A,11.89993,N/A,6.750015,N/A,N/A
2,ABT,110.49,N/A,37.45,N/A,5.09,N/A,4.77,N/A,24.938352,N/A,9.071074,N/A,N/A
3,ABBV,154.87,N/A,42.43,N/A,23.05,N/A,5.06,N/A,12.976413,N/A,8.997987,N/A,N/A
4,ACN,352.80,N/A,32.73,N/A,8.84,N/A,3.65,N/A,18.632979,N/A,10.899937,N/A,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,YUM,130.94,N/A,24.89,N/A,-4.48,N/A,5.20,N/A,19.793725,N/A,8.980833,N/A,N/A
495,ZBRA,275.60,N/A,30.72,N/A,4.70,N/A,2.79,N/A,18.78035,N/A,6.93918,N/A,N/A
496,ZBH,121.71,N/A,53.85,N/A,2.02,N/A,3.48,N/A,15.912433,N/A,5.850316,N/A,N/A
497,ZION,44.45,N/A,4.74,N/A,1.35,N/A,1.32,N/A,3.631114,N/A,1.370346,N/A,N/A


In [12]:
# Calculating Percentiles

from_cols = ['Price-to-Earnings Ratio',
             'Price-to-Book Ratio', 'Price-to-Sales Ratio',
             'EV/EBITDA', 'EV/GP', 'RV Score']

to_cols = ['PE Percentile',
         'PB Percentile', 'PS Percentile',
         'EV/EBITDA Percentile','EV/GP Percentile']


for row in rv_df_cleaned.index:
    for change_col, percentile_col in zip(from_cols, to_cols):
        rv_df_cleaned.loc[row, percentile_col] = stats.percentileofscore(rv_df_cleaned[change_col], rv_df_cleaned.loc[row, change_col])

rv_df_cleaned

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,109.15,N/A,-8.16,4.90982,12.86,90.180361,1.83,35.270541,-9.34938,1.202405,4.933437,31.462926,N/A
1,AOS,82.36,N/A,41.81,82.56513,5.36,69.539078,2.66,50.901804,11.89993,35.871743,6.750015,47.294589,N/A
2,ABT,110.49,N/A,37.45,78.156313,5.09,66.533066,4.77,71.442886,24.938352,84.96994,9.071074,64.128257,N/A
3,ABBV,154.87,N/A,42.43,82.765531,23.05,95.390782,5.06,73.947896,12.976413,41.482966,8.997987,62.124248,N/A
4,ACN,352.80,N/A,32.73,72.144289,8.84,82.965932,3.65,61.923848,18.632979,69.138277,10.899937,75.551102,N/A
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,YUM,130.94,N/A,24.89,53.707415,-4.48,6.212425,5.20,75.450902,19.793725,73.547094,8.980833,61.923848,N/A
495,ZBRA,275.60,N/A,30.72,68.136273,4.70,64.428858,2.79,53.106212,18.78035,69.539078,6.93918,49.298597,N/A
496,ZBH,121.71,N/A,53.85,88.777555,2.02,31.362725,3.48,59.318637,15.912433,54.90982,5.850316,39.679359,N/A
497,ZION,44.45,N/A,4.74,7.214429,1.35,16.032064,1.32,21.943888,3.631114,2.805611,1.370346,7.815631,N/A


In [13]:
# Calculating RV score

for row in rv_df_cleaned.index:
    rv_df_cleaned.loc[row, 'RV Score'] = np.mean(rv_df_cleaned[to_cols].loc[row])
rv_df_cleaned

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
0,MMM,109.15,N/A,-8.16,4.90982,12.86,90.180361,1.83,35.270541,-9.34938,1.202405,4.933437,31.462926,32.60521
1,AOS,82.36,N/A,41.81,82.56513,5.36,69.539078,2.66,50.901804,11.89993,35.871743,6.750015,47.294589,57.234469
2,ABT,110.49,N/A,37.45,78.156313,5.09,66.533066,4.77,71.442886,24.938352,84.96994,9.071074,64.128257,73.046092
3,ABBV,154.87,N/A,42.43,82.765531,23.05,95.390782,5.06,73.947896,12.976413,41.482966,8.997987,62.124248,71.142285
4,ACN,352.80,N/A,32.73,72.144289,8.84,82.965932,3.65,61.923848,18.632979,69.138277,10.899937,75.551102,72.344689
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
494,YUM,130.94,N/A,24.89,53.707415,-4.48,6.212425,5.20,75.450902,19.793725,73.547094,8.980833,61.923848,54.168337
495,ZBRA,275.60,N/A,30.72,68.136273,4.70,64.428858,2.79,53.106212,18.78035,69.539078,6.93918,49.298597,60.901804
496,ZBH,121.71,N/A,53.85,88.777555,2.02,31.362725,3.48,59.318637,15.912433,54.90982,5.850316,39.679359,54.809619
497,ZION,44.45,N/A,4.74,7.214429,1.35,16.032064,1.32,21.943888,3.631114,2.805611,1.370346,7.815631,11.162325


## Taking top 50 companies with the best RV score

In [14]:
best_rv = rv_df_cleaned.sort_values('RV Score', ascending=True)
best_rv = best_rv[best_rv['Price-to-Earnings Ratio']>5][:50] # Removing companies with very bad earnings
best_rv

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
25,AAL,13.980,N/A,5.22,7.815631,-1.7800,6.613226,0.1728,0.801603,4.802942,5.210421,0.6057,1.202405,4.328657
215,GM,36.155,N/A,5.09,7.61523,0.6634,7.61523,0.2873,2.40481,1.33873,1.603206,0.945548,3.607214,4.569138
112,C,51.600,N/A,8.23,12.224449,0.5183,7.014028,0.6666,8.416834,4.341563,3.607214,0.701729,1.803607,6.613226
200,F,12.350,N/A,8.07,11.422846,1.1000,10.821643,0.2796,2.004008,2.518094,1.803607,1.477651,8.817635,6.973948
424,SYF,38.315,N/A,7.10,9.018036,1.2200,13.426854,0.7425,9.218437,4.474034,4.008016,0.991094,4.008016,7.935872
290,L,69.590,N/A,11.90,20.04008,1.0700,10.420842,0.9969,14.228457,4.102126,3.406814,0.996934,4.408818,10.501002
113,CFG,33.550,N/A,8.30,12.42485,0.7517,8.016032,1.3300,22.244489,5.02362,5.811623,1.410408,8.216433,11.342685
180,EG,353.510,N/A,6.47,8.817635,1.3600,16.432866,1.0700,16.332665,6.213312,10.621242,1.070079,4.809619,11.402806
28,AIG,67.905,N/A,13.19,23.446894,1.1900,12.825651,0.9708,13.226453,4.654622,4.809619,0.970759,3.807615,11.623246
137,CVS,79.035,N/A,11.94,20.240481,1.3600,16.432866,0.2916,2.60521,7.44695,13.627255,1.078864,5.210421,11.623246


## Function which shows minimum budget for buying all positions in dataframe

In [15]:
def min_budget(df):
    minimum = math.ceil(np.sum(df['Price']))
    print(f'To own the above portfolio with 1 stock per company you need at least ${minimum}')
    
min_budget(best_rv)

To own the above portfolio with 1 stock per company you need at least $4597


## Calculating the number of shares with the same monetary amount per company

In [16]:
def calc_num_of_shares(df):

    while True:
        try:
            whole_amount = int(input('Input your portfolio size in dollars -> '))
            break
        except:
            print('Do not input text, only numbers!\n')
    
    amount_on_ticker = whole_amount/len(df.index)
    df['Number of Shares to Buy'] = [math.floor(amount_on_ticker/price) for price in df['Price']]


calc_num_of_shares(best_rv)
best_rv

,Ticker,Price,Number of Shares to Buy,Price-to-Earnings Ratio,PE Percentile,Price-to-Book Ratio,PB Percentile,Price-to-Sales Ratio,PS Percentile,EV/EBITDA,EV/EBITDA Percentile,EV/GP,EV/GP Percentile,RV Score
25,AAL,13.980,1430,5.22,7.815631,-1.7800,6.613226,0.1728,0.801603,4.802942,5.210421,0.6057,1.202405,4.328657
215,GM,36.155,553,5.09,7.61523,0.6634,7.61523,0.2873,2.40481,1.33873,1.603206,0.945548,3.607214,4.569138
112,C,51.600,387,8.23,12.224449,0.5183,7.014028,0.6666,8.416834,4.341563,3.607214,0.701729,1.803607,6.613226
200,F,12.350,1619,8.07,11.422846,1.1000,10.821643,0.2796,2.004008,2.518094,1.803607,1.477651,8.817635,6.973948
424,SYF,38.315,521,7.10,9.018036,1.2200,13.426854,0.7425,9.218437,4.474034,4.008016,0.991094,4.008016,7.935872
290,L,69.590,287,11.90,20.04008,1.0700,10.420842,0.9969,14.228457,4.102126,3.406814,0.996934,4.408818,10.501002
113,CFG,33.550,596,8.30,12.42485,0.7517,8.016032,1.3300,22.244489,5.02362,5.811623,1.410408,8.216433,11.342685
180,EG,353.510,56,6.47,8.817635,1.3600,16.432866,1.0700,16.332665,6.213312,10.621242,1.070079,4.809619,11.402806
28,AIG,67.905,294,13.19,23.446894,1.1900,12.825651,0.9708,13.226453,4.654622,4.809619,0.970759,3.807615,11.623246
137,CVS,79.035,253,11.94,20.240481,1.3600,16.432866,0.2916,2.60521,7.44695,13.627255,1.078864,5.210421,11.623246


## Saving dataframe to excel

In [17]:
# Create ExcelWriter object
writer = pd.ExcelWriter('data/value_strategy.xlsx', engine='xlsxwriter')

# Write DataFrame to Excel
best_rv.to_excel(writer, sheet_name='Value Strategy', index=False)

background_color = '#d3d3d3' 

string_format = writer.book.add_format(
    {
        'bg_color':background_color,
        'border':1
    }
)
dollar_format_long = writer.book.add_format(
    {   
        'num_format':'$0.00',
        'bg_color':background_color,
        'border':1
    }
)

integer_format = writer.book.add_format(
    {   
        'num_format':'0',
        'bg_color':background_color,
        'border':1
    }
)

percentile_format = writer.book.add_format(
    {   
        'num_format':'0"th"',
        'bg_color':background_color,
        'border':1
    }
) 

float_format = writer.book.add_format(
    {   
        'num_format':'0.00',
        'bg_color':background_color,
        'border':1
    }
) 

header_format = writer.book.add_format(
    {   
        'bg_color':'#008080',
        'border':1,
        'bold':True
    }
)



latin_letters_dict = {num: chr(num + 65) for num in range(26)}

col_format_dict = {
    'Ticker':string_format,
    'Price':dollar_format_long,
    'Number of Shares to Buy':integer_format,
    'Price-to-Earnings Ratio':float_format,
    'PE Percentile':percentile_format,
    'Price-to-Book Ratio':float_format,
    'PB Percentile':percentile_format,
    'Price-to-Sales Ratio':float_format,
    'PS Percentile':percentile_format,
    "EV/EBITDA":float_format,
    'EV/EBITDA Percentile':percentile_format,
    'EV/GP': float_format,
    'EV/GP Percentile': percentile_format,
    'RV Score': float_format
}


for i, col in enumerate(best_rv.columns):
    letter = latin_letters_dict[i]
    writer.sheets['Value Strategy'].write(f'{letter}1', col, header_format)
    writer.sheets['Value Strategy'].set_column(f'{letter}:{letter}', 23)
    writer.sheets['Value Strategy'].conditional_format(1, i, len(best_rv)+1, i, 
                                                          {'type': 'no_blanks', 'format': col_format_dict[col]})
    

writer.close()